In [2]:
#!/usr/bin/env python3
#_*_ coding:utf-8 _*_

import numpy as np
import struct

# 读取文件
train_images_idx3_ubyte_file = './MNIST/train-images-idx3-ubyte'
train_labels_idx1_ubyte_file = './MNIST/train-labels-idx1-ubyte'

test_images_idx3_ubyte_file = './MNIST/t10k-images-idx3-ubyte'
test_labels_idx1_ubyte_file = './MNIST/t10k-labels-idx1-ubyte'

# 数据集里的图片是二进制的,转化成numpy需要的数据形式
def decode_idx3_ubyte(idx3_ubyte_file):
    bin_data = open(idx3_ubyte_file, 'rb').read()   # 数据集里的图片是二进制

    offset = 0
    fmt_header = '>IIII'    # f='>IIII'是二进制大端读取方法
    # 文件协议的描述
    magic_number, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, bin_data, offset)
    print("magic:%d, count: %d, size: %d*%d" % (magic_number, num_images, num_rows, num_cols))

    image_size = num_rows * num_cols
    offset += struct.calcsize(fmt_header)
    fmt_image = '>' + str(image_size) + 'B'
    images = np.empty((num_images, num_rows, num_cols))
    for i in range(num_images):
        if (i + 1) % 10000 == 0:
            print("done %d" % (i + 1) + "pictures")
        images[i] = np.array(struct.unpack_from(fmt_image, bin_data, offset)).reshape((num_rows, num_cols))
        offset += struct.calcsize(fmt_image)
    return images

# 标签 从二进制转换成numpy所需的形式
def decode_idx1_ubyte(idx1_ubyte_file):
    bin_data = open(idx1_ubyte_file, 'rb').read()

    offset = 0
    fmt_header = '>ii'
    magic_number, num_images = struct.unpack_from(fmt_header, bin_data, offset)
    print("magic:%d, num_images: %d pairs" % (magic_number, num_images))

    offset += struct.calcsize(fmt_header)
    fmt_image = '>B'
    labels = np.empty(num_images)
    for i in range(num_images):
        if (i + 1) % 10000 == 0:
            print("done %d" % (i + 1) + "pairs")
        labels[i] = struct.unpack_from(fmt_image, bin_data, offset)[0]
        offset += struct.calcsize(fmt_image)
    return labels

# 载入数据
def load_train_images(idx_ubyte_file=train_images_idx3_ubyte_file):
    return decode_idx3_ubyte(idx_ubyte_file)

def load_train_labels(idx_ubyte_file=train_labels_idx1_ubyte_file):
    return decode_idx1_ubyte(idx_ubyte_file)

def load_test_images(idx_ubyte_file=test_images_idx3_ubyte_file):
    return decode_idx3_ubyte(idx_ubyte_file)

def load_test_labels(idx_ubyte_file=test_labels_idx1_ubyte_file):
    return decode_idx1_ubyte(idx_ubyte_file)

# 标准正则化。以及初始化参数。这里用的是均值初始化。
def narmalize_data(ima):
    a_max = np.max(ima)
    a_min = np.min(ima)
    for j in range(ima.shape[0]):
        ima[j] = (ima[j] - a_min) / (a_max - a_min)
    return ima

def initialize_with_zeros(n_x, n_h, n_y):
    np.random.seed(2)
    W1 = np.random.uniform(-np.sqrt(6) / np.sqrt(n_x + n_h), np.sqrt(6) / np.sqrt(n_h + n_x), size=(n_h, n_x))
    b1 = np.zeros((n_h, 1))
    W2 = np.random.uniform(-np.sqrt(6) / np.sqrt(n_y + n_h), np.sqrt(6) / np.sqrt(n_y + n_h), size=(n_y, n_h))
    b2 = np.zeros((n_y, 1))

    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}

    return parameters

# 前向传播和损失函数的计算
def forward_propagation(X, parameters):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    return A2, cache


def costloss(A2, Y, parameters):
    t = 0.00000000001
    logprobs = np.multiply(np.log(A2 + t), Y) + np.multiply(np.log(1 - A2 + t), (1 - Y))

    cost = np.sum(logprobs, axis=0, keepdims=True) / A2.shape[0]
    return cost

# 反向传播和更新参数
def back_propagation(parameters, cache, X, Y):

    W1 = parameters["W1"]
    W2 = parameters["W2"]
    A1 = cache["A1"]
    A2 = cache["A2"]
    Z1 = cache["Z1"]

    dZ2 = A2 - Y
    dW2 = np.dot(dZ2, A1.T)
    db2 = np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.dot(W2.T, dZ2) * (1 - np.power(A1, 2))
    dW1 = np.dot(dZ1, X.T)
    db1 = np.sum(dZ1, axis=1, keepdims=True)
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    return grads


def update_para(parameters, grads, learning_rate):
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    return parameters

# 定义sigmoid激活函数，softmax等参数。image2vector的作用是将输入从28*28的图片变成一个列向量。
def sigmoid(x):
    s=1/(1+np.exp(-x))
    return s
def image2vector(image):
    v=np.reshape(image,[784,1])
    return v
def softmax(x):
    v=np.argmax(x)
    return v

# n_x为输入图片的大小，本次为28*28。n_h为神经网络的层数。本次实验设计的是两层的神经网络。一个中间层，一个输出层。（输入层不算入层数）。
# 本次试验中，中间层是32个神经元，选别的数量也行。此时正确识别数量为9119个。若选为10个神经元，实验得到正确识别结果为8619.可见准确率下降了。
# n_y是输出神经元的数量。本次因为是从0-9共10个数字，所以n_y=10。ii是测试集中正确识别了的图片数。
# 然后开始训练，循环。直至结束。
if __name__ == '__main__':
    train_images = load_train_images()
    train_labels = load_train_labels()
    test_images = load_test_images()
    test_labels = load_test_labels()

    ii = 0
    n_x = 28 * 28
    n_h = 32
    n_y = 10
    parameters = initialize_with_zeros(n_x, n_h, n_y)
    train_step = 50000
    # train_step = 2000
    for i in range(train_step):
        img_train = train_images[i]
        label_train1 = train_labels[i]
        label_train = np.zeros((10, 1))
        ttt = 0.001
        if i > 1000:
            ttt = ttt * 0.999
        label_train[int(train_labels[i])] = 1
        imgvector1 = image2vector(img_train)
        imgvector = narmalize_data(imgvector1)
        A2, cache = forward_propagation(imgvector, parameters)
        pre_label = softmax(A2)

        costl = costloss(A2, label_train, parameters)
        grads = back_propagation(parameters, cache, imgvector, label_train)
        parameters = update_para(parameters, grads, learning_rate=ttt)
        grads["dW1"] = 0
        grads["dW2"] = 0
        grads["db1"] = 0
        grads["db2"] = 0
        if i%1000 == 0:
            print("cost after iteration %i:" % (i))
            print(costl)
    test_steps = 10000
    for i in range(test_steps):
        img_train = test_images[i]
        vector_image = narmalize_data(image2vector(img_train))
        label_trainx = test_labels[i]
        aa2, xxx = forward_propagation(vector_image, parameters)
        predict_value = softmax(aa2)
        if predict_value == int(label_trainx):
            ii = ii + 1
        if i%1000:
            print("test %d steps" % i)
    print(np.float32(ii/test_steps))    # 识别率

magic:2051, count: 60000, size: 28*28
done 10000pictures
done 20000pictures
done 30000pictures
done 40000pictures
done 50000pictures
done 60000pictures
magic:2049, num_images: 60000 pairs
done 10000pairs
done 20000pairs
done 30000pairs
done 40000pairs
done 50000pairs
done 60000pairs
magic:2051, count: 10000, size: 28*28
done 10000pictures
magic:2049, num_images: 10000 pairs
done 10000pairs
cost after iteration 0:
[[-0.62811176]]
cost after iteration 1000:
[[-0.25876455]]
cost after iteration 2000:
[[-0.34338076]]
cost after iteration 3000:
[[-0.36470343]]
cost after iteration 4000:
[[-0.13882801]]
cost after iteration 5000:
[[-0.1216571]]
cost after iteration 6000:
[[-0.09286688]]
cost after iteration 7000:
[[-0.26100521]]
cost after iteration 8000:
[[-0.06171823]]
cost after iteration 9000:
[[-0.13944045]]
cost after iteration 10000:
[[-0.04790998]]
cost after iteration 11000:
[[-0.05128911]]
cost after iteration 12000:
[[-0.09299672]]
cost after iteration 13000:
[[-0.10173517]]
cost 

KeyboardInterrupt: 